<a href="https://colab.research.google.com/github/eduardoplima/classificacao-satelite/blob/main/classificacao-satelite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificação de Imagens de Satélite

A classificação de imagens de satélite é uma tarefa crucial em diversas áreas, como monitoramento ambiental, planejamento urbano, agricultura de precisão e análise de desastres naturais. Essas imagens contêm informações valiosas sobre o uso da terra, cobertura vegetal e infraestrutura, que podem ser extraídas para criar mapas detalhados, identificar padrões e tomar decisões informadas. Tradicionalmente, a classificação de imagens de satélite exigia técnicas de processamento de imagem baseadas em regras ou algoritmos estatísticos. No entanto, com os avanços da Visão Computacional e a popularização do Deep Learning, novas abordagens baseadas em redes neurais convolucionais (CNNs) têm demonstrado um desempenho impressionante em termos de precisão e capacidade de generalização. As CNNs são especialmente eficazes para tarefas como a classificação de imagens de satélite, pois são capazes de aprender características relevantes das imagens sem a necessidade de engenharia manual de características.

Uma abordagem recente que tem ganhado destaque para melhorar a performance de redes neurais em tarefas de classificação de imagens de satélite é o Transfer Learning. Essa técnica permite utilizar modelos previamente treinados em grandes conjuntos de dados (como o ImageNet) e ajustá-los para tarefas específicas, como a classificação de imagens de satélite. Modelos como o MobileNetV2, EfficientNetB0 e DenseNet121 são exemplos de arquiteturas que se destacam nesse contexto. O MobileNetV2 é uma rede neural eficiente e leve, ideal para dispositivos móveis e aplicações que requerem baixo custo computacional. O EfficientNetB0, por sua vez, é otimizado para obter a melhor relação entre acurácia e eficiência, oferecendo alto desempenho com um número reduzido de parâmetros. Já o DenseNet121 se caracteriza por um design de rede densa, onde cada camada é conectada a todas as camadas anteriores, facilitando a reutilização de características e permitindo melhores resultados em tarefas complexas. Essas arquiteturas, quando combinadas com o Transfer Learning, são extremamente poderosas para a classificação de imagens de satélite, permitindo resultados precisos mesmo com limitações de dados e recursos computacionais.

# Configuração

Nessa seção configuramos nosso ambiente e importamos os pacotes necessários para o código a seguir

In [2]:
#!pip install tensorflow==2.13.0

In [3]:
#!pip install -q lightgbm

In [4]:
#!pip install -q kaggle
#!pip install -q timm torch torchvision matplotlib scikit-learn


In [5]:
# Bibliotecas padrão
import os
import random


# Machine Learning e Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model

import torch
import timm
import torchvision.transforms as T
from torch.utils.data import DataLoader
from torchvision import datasets

# Análise de dados e visualização
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import lightgbm as lgb


# Utilidades
from PIL import Image
from google.colab import files

# O dataset UC Merced Land Use Dataset

O dataset utilizado nesse trabalho é o UC Merced Land Use Dataset. Trata-se de uma coleção de imagens de alta resolução capturadas por sensores remotos, projetada para pesquisas em classificação de uso da terra. Composta por 21 categorias distintas — como agrícola, aeroporto, campo de beisebol, praia, edifícios, floresta, rodovia, entre outras —, cada classe contém 100 imagens de 256×256 pixels. Essas imagens foram extraídas manualmente de grandes imagens da coleção Urban Area Imagery do USGS National Map, com resolução espacial de 1 pé (aproximadamente 30 cm por pixel), cobrindo diversas áreas urbanas dos Estados Unidos.

Nas células a seguir fazemos o download do dataset a partir de um repositório Kaggle e convertemos as imagens para um formato legível pelo Tensorflow. Também exibimos algumas imagens aleatórias de teste.

In [ ]:
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d abdulhasibuddin/uc-merced-land-use-dataset

In [ ]:
!unzip -q uc-merced-land-use-dataset.zip -d uc-merced-land-use-dataset

In [ ]:
for folder in os.listdir('uc-merced-land-use-dataset/UCMerced_LandUse/Images'):
    print(folder, "→", len(os.listdir(f"uc-merced-land-use-dataset/UCMerced_LandUse/Images/{folder}")), "imagens")

In [ ]:

source_dir = "uc-merced-land-use-dataset/UCMerced_LandUse/Images"
destination_dir = "uc-merced-land-use-dataset/UCMerced_LandUse/ImagesJpg"

for folder in os.listdir(source_dir):
    folder_path = os.path.join(source_dir, folder)
    if os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
            if filename.endswith(".tif"):
                source_path = os.path.join(folder_path, filename)
                name, ext = os.path.splitext(filename)
                destination_path = os.path.join(folder_path, name + ".jpg")

                try:
                    img = Image.open(source_path)
                    img.save(destination_path, "JPEG")
                except IOError as e:
                    print(f"Erro convertendo {source_path}: {e}")


In [ ]:
for folder in os.listdir('uc-merced-land-use-dataset/UCMerced_LandUse/Images'):
  path = f"uc-merced-land-use-dataset/UCMerced_LandUse/Images/{folder}"
  image = Image.open(os.path.join(path, random.choice(os.listdir(path))))
  plt.imshow(image)
  plt.title(folder)
  plt.axis('off')
  plt.show()

# Criação de datasets de treinamento, validação e teste

In [ ]:
dataset_path = 'uc-merced-land-use-dataset/UCMerced_LandUse/Images'
classes_name = [folder for folder in os.listdir(dataset_path) if os.path.isdir(os.path.join('uc-merced-land-use-dataset/UCMerced_LandUse/Images', folder))]

batch_size = 32
img_height = 256
img_width = 256
seed = 123

train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.3,
    subset="training",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.3,
    subset="validation",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_batches = tf.data.experimental.cardinality(val_ds)
test_ds = val_ds.take(val_batches // 2)
val_ds = val_ds.skip(val_batches // 2)

print(f"Training batches: {tf.data.experimental.cardinality(train_ds).numpy()}")
print(f"Validation batches: {tf.data.experimental.cardinality(val_ds).numpy()}")
print(f"Test batches: {tf.data.experimental.cardinality(test_ds).numpy()}")

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)


num_classes = 21


translated_classes_names = [
 'agrícola',
 'avião',
 'campo de beisebol',
 'praia',
 'edifícios',
 'vegetação arbustiva',
 'área residencial densa',
 'floresta',
 'autoestrada',
 'campo de golfe',
 'porto',
 'cruzamento',
 'área residencial média',
 'parque de casas móveis',
 'viaduto',
 'estacionamento',
 'rio',
 'pista de pouso/decolagem',
 'área residencial esparsa',
 'tanques de armazenamento',
 'quadra de tênis'
]


# Implementação de uma rede neural convolucional simples

In [ ]:
simple_model = keras.Sequential([
    layers.Rescaling(1./255, input_shape=(img_height,
                                          img_width,
                                          3)),

    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),

    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),

    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])

simple_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.\
    SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

epochs = 50
history = simple_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

# 8. Evaluate on the Test set
test_loss, test_acc = simple_model.evaluate(test_ds)
print('\nTest accuracy:', test_acc)

# 9. Plot training results (optional)
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(14,5))
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
# Initialize predictions
y_true = []
y_pred = []

# Get predictions from the model
for images, labels in test_ds:
    preds = simple_model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))


# Compute metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

# Create or append to a DataFrame
results_df = pd.DataFrame(columns=["Model", "Accuracy", "Precision", "Recall", "F1-Score"])
results_df.loc[len(results_df)] = ["CNN Simples", accuracy, precision, recall, f1]

In [ ]:
results_df

In [ ]:
cm = confusion_matrix(y_true, y_pred)
class_names = os.listdir('uc-merced-land-use-dataset/UCMerced_LandUse/Images')
cmd = ConfusionMatrixDisplay(cm, display_labels=class_names)

plt.figure(figsize=(15, 15))
cmd.plot(cmap=plt.cm.Blues, values_format='d')
plt.title("Matriz de Confusão - CNN Simples")
plt.xticks(rotation=90)
plt.xlabel("Classe Predita")
plt.ylabel("Classe Verdadeira")
plt.show()


In [ ]:
results_df

# Uso de Transfer Learning para implementação de Rede Neural MobileNetV2

In [ ]:

# 2. Data augmentation
data_augmentation = keras.Sequential([
    layers.RandomFlip('horizontal_and_vertical'),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.1),
])

# 3. Load the base model
base_mobilenet_model = keras.applications.MobileNetV2(
    input_shape=(256, 256, 3),
    include_top=False,
    weights='imagenet'
)
base_mobilenet_model.trainable = False  # Freeze the base model initially

# 4. Create the full model
inputs = keras.Input(shape=(256, 256, 3))
x = data_augmentation(inputs)
x = keras.applications.mobilenet_v2.preprocess_input(x)
x = base_mobilenet_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(21, activation='softmax')(x)

mobilenet_model = keras.Model(inputs, outputs)

# 5. Compile
mobilenet_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 6. Train
epochs = 50
history = mobilenet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

# 7. Fine-tuning (optional but recommended)
base_mobilenet_model.trainable = True
# Recompile with lower learning rate
mobilenet_model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
fine_tune_epochs = 50
history_fine = mobilenet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=fine_tune_epochs
)

# 8. Evaluate
test_loss, test_acc = mobilenet_model.evaluate(test_ds)
print('\nTest accuracy after fine-tuning:', test_acc)


In [ ]:
# Initialize predictions
y_true = []
y_pred = []

# Get predictions from the model
for images, labels in test_ds:
    preds = mobilenet_model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))


# Compute metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

# Create or append to a DataFrame
results_df.loc[len(results_df)] = ["MobileNetV2", accuracy, precision, recall, f1]

In [ ]:
results_df

In [ ]:
cm = confusion_matrix(y_true, y_pred)
class_names = os.listdir('uc-merced-land-use-dataset/UCMerced_LandUse/Images')
cmd = ConfusionMatrixDisplay(cm, display_labels=class_names)

plt.figure(figsize=(15, 15))
cmd.plot(cmap=plt.cm.Blues, values_format='d')
plt.title("Matriz de Confusão")
plt.xticks(rotation=90)
plt.show()


# Uso de Transfer Learning para implementação de Rede Neural EfficientNetB0

In [ ]:


# 2. Data Augmentation
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
])

# 3. Load EfficientNetB0
base_efficientnetb0_model = keras.applications.EfficientNetB0(
    input_shape=(256, 256, 3),
    include_top=False,
    weights='imagenet'
)
base_efficientnetb0_model.trainable = False  # Freeze at first

# 4. Build the model
inputs = keras.Input(shape=(256, 256, 3))
x = data_augmentation(inputs)
x = keras.applications.efficientnet.preprocess_input(x)
x = base_efficientnetb0_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(21, activation='softmax')(x)

efficientnetb0_model = keras.Model(inputs, outputs)

# 5. Compile
efficientnetb0_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 6. Train
epochs = 50
history = efficientnetb0_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

# 7. Fine-tune (optional, improves results)
base_efficientnetb0_model.trainable = True
efficientnetb0_model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
fine_tune_epochs = 50
history_fine = efficientnetb0_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=fine_tune_epochs
)

# 8. Evaluate
test_loss, test_acc = efficientnetb0_model.evaluate(test_ds)
print("\nTest accuracy:", test_acc)


In [ ]:
# Initialize predictions
y_true = []
y_pred = []

# Get predictions from the model
for images, labels in test_ds:
    preds = efficientnetb0_model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))


# Compute metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

# Create or append to a DataFrame
results_df.loc[len(results_df)] = ["EfficientNetB0", accuracy, precision, recall, f1]

In [ ]:
results_df

In [ ]:
for images, labels in test_ds:
    preds = efficientnetb0_model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))


cm = confusion_matrix(y_true, y_pred)
class_names = os.listdir('uc-merced-land-use-dataset/UCMerced_LandUse/Images')
cmd = ConfusionMatrixDisplay(cm, display_labels=class_names)

plt.figure(figsize=(15, 15))
cmd.plot(cmap=plt.cm.Blues, values_format='d')
plt.title("Confusion Matrix")
plt.xticks(rotation=90)
plt.show()


# Uso de Transfer Learning para implementação de Rede Neural DenseNet121

In [ ]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
])

base_densenet121_model = keras.applications.DenseNet121(
    input_shape=(256, 256, 3),
    include_top=False,
    weights='imagenet'
)
base_densenet121_model.trainable = False

inputs = keras.Input(shape=(256, 256, 3))
x = data_augmentation(inputs)
x = keras.applications.densenet.preprocess_input(x)
x = base_densenet121_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(21, activation='softmax')(x)

densenet121_model = keras.Model(inputs, outputs)

densenet121_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

epochs = 50
history = densenet121_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

base_densenet121_model.trainable = True
densenet121_model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
fine_tune_epochs = 50
history_fine = densenet121_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=fine_tune_epochs
)

test_loss, test_acc = densenet121_model.evaluate(test_ds)
print("\nTest accuracy after fine-tuning:", test_acc)


In [ ]:
# Initialize predictions
y_true = []
y_pred = []

# Get predictions from the model
for images, labels in test_ds:
    preds = efficientnetb0_model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))


# Compute metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

# Create or append to a DataFrame
results_df.loc[len(results_df)] = ["DenseNet121", accuracy, precision, recall, f1]
results_df

In [ ]:

for images, labels in test_ds:
    preds = densenet121_model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

# 2. Create Confusion Matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true, y_pred)
class_names = os.listdir('uc-merced-land-use-dataset/UCMerced_LandUse/Images')
cmd = ConfusionMatrixDisplay(cm, display_labels=class_names)

plt.figure(figsize=(15, 15))
cmd.plot(cmap=plt.cm.Blues, values_format='d')
plt.title("Matriz de Confusão - DenseNet 121")
plt.xticks(rotation=90)
plt.xlabel("Classe Predita")
plt.ylabel("Classe Verdadeira")
plt.show()

# Uso de Transformers visuais para obtenção de embedding do dataset

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset_path = 'uc-merced-land-use-dataset/UCMerced_LandUse/Images'

transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# 5. Load dataset
full_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Optional: split into train/val/test (simple random split)
train_size = int(0.7 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size, test_size])

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

class_names = full_dataset.classes
print("Classes:", class_names)

# 6. Load ViT
vit_model = timm.create_model('vit_base_patch16_224', pretrained=True)
vit_model.head = torch.nn.Identity()  # Remove classification head
vit_model = vit_model.to(device)
vit_model.eval()

# 7. Extract embeddings
all_embeddings = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)

        embeddings = vit_model(images)  # Directly pass images
        all_embeddings.append(embeddings.cpu())
        all_labels.append(labels.cpu())

# Stack results
all_embeddings = torch.cat(all_embeddings).numpy()
all_labels = torch.cat(all_labels).numpy()

print(f"Embeddings shape: {all_embeddings.shape}")

# 8. Dimensionality Reduction (PCA + t-SNE)
pca = PCA(n_components=50)
pca_result = pca.fit_transform(all_embeddings)

tsne = TSNE(n_components=2, perplexity=30, n_iter=3000, random_state=42)
tsne_result = tsne.fit_transform(pca_result)




In [ ]:
all_embeddings[0].shape

In [ ]:
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np

# 1. Define custom distinct colors again
custom_colors = [
    '#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00',
    '#ffff33', '#a65628', '#f781bf', '#999999', '#66c2a5',
    '#fc8d62', '#8da0cb', '#e78ac3', '#a6d854', '#ffd92f',
    '#e5c494', '#b3b3b3', '#1b9e77', '#d95f02', '#7570b3',
    '#e7298a'
]

# 2. Plot t-SNE embeddings
plt.figure(figsize=(16, 9))  # Slide format 16:9

colors_for_points = [custom_colors[label] for label in all_labels]

scatter = plt.scatter(
    tsne_result[:, 0],
    tsne_result[:, 1],
    c=colors_for_points,
    alpha=0.8
)

# 3. Add annotations: annotate the mean position of each class
for i, class_name in enumerate(translated_classes_names):
    idxs = np.where(all_labels == i)
    if len(idxs[0]) > 0:
        mean_x = np.mean(tsne_result[idxs, 0])
        mean_y = np.mean(tsne_result[idxs, 1])
        plt.text(
            mean_x,
            mean_y,
            class_name,
            fontsize=8,
            weight='bold',
            ha='center',
            va='center',
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='black', boxstyle='round,pad=0.2')
        )

# 4. Custom Legend — two columns
handles = [
    mpatches.Patch(color=custom_colors[i], label=translated_classes_names[i])
    for i in range(len(translated_classes_names))
]

plt.legend(
    handles=handles,
    bbox_to_anchor=(1.05, 1),
    loc='upper left',
    borderaxespad=0.,
    title='Classes',
    fontsize='small',
    ncol=2
)

plt.title('Visualização dos Embeddings do UC Merced', fontsize=16)
plt.xlabel('Dimensão 1')
plt.ylabel('Dimensão 2')

plt.tight_layout()

# 5. Save
plt.savefig('uc_merced_tsne_distinct_colors_annotated.png', dpi=300)
plt.savefig('uc_merced_tsne_distinct_colors_annotated.pdf')

# 6. Show
plt.show()


# Uso de embedding para treinamento de modelos de aprendizado de máquina

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 1. Dados de entrada
X = all_embeddings  # Embeddings extraídos do Vision Transformer
y = all_labels      # Rótulos correspondentes

# 2. Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Train size: {X_train.shape}, Test size: {X_test.shape}")

# 3. Definição dos modelos
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Support Vector Machine (SVM)": SVC(kernel='rbf', probability=True),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "K-Nearest Neighbors (KNN)": KNeighborsClassifier(n_neighbors=5),
    "Gradient Boosting": lgb.LGBMClassifier()
}

# 5. Treinamento e avaliação dos modelos
for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_test, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)

    results_df.loc[len(results_df)] = [name, acc, precision, recall, f1]
    print(f"Accuracy on Test Set: {acc:.4f}")

# 6. Exibição do DataFrame
print("\nResumo das Métricas:")
display(results_df)


# Avaliação e Visualização dos resultados

In [ ]:
# prompt: create a visualization of the results_df dataframe

import matplotlib.pyplot as plt

# Assuming results_df is already defined as in your provided code

results_df.plot(x="Model", y=["Accuracy", "Precision", "Recall", "F1-Score"], kind="bar", figsize=(10, 6))
plt.title("Performance dos Modelos")
plt.ylabel("Score")
plt.xticks(rotation=45)
plt.legend(title="Métricas")
plt.tight_layout()
plt.show()
